In [7]:
import random
from dataclasses import dataclass
from typing import List, Tuple

# --- 定数定義 ---
WIDTH, HEIGHT = 10, 10        # 迷路のセル数
START, GOAL   = (1, 1), (8, 8) # 座標系は (x, y)

# --- データ構造 ---
@dataclass
class Maze:
    vert_walls: List[List[bool]]      # 垂直壁 (y, x)  長さ: HEIGHT × (WIDTH-1)
    horiz_walls: List[List[bool]]     # 水平壁 (y, x)  長さ: (HEIGHT-1) × WIDTH

def generate_maze() -> Maze:
    """再帰的バックトラッキングで木構造を生成し、壁の有無で保持する。"""
    visited = [[False]*WIDTH for _ in range(HEIGHT)]
    vert   = [[True]*(WIDTH-1) for _ in range(HEIGHT)]     # True = 壁あり
    horiz  = [[True]*WIDTH      for _ in range(HEIGHT-1)]

    stack: List[Tuple[int, int]] = [START]
    visited[START[1]][START[0]] = True                    # y, x の順に注意

    while stack:
        x, y = stack[-1]

        # 未訪問の隣接セルを列挙
        candidates = []
        if x > 0           and not visited[y][x-1]: candidates.append(('L', x-1, y))
        if x < WIDTH-1     and not visited[y][x+1]: candidates.append(('R', x+1, y))
        if y > 0           and not visited[y-1][x]: candidates.append(('U', x, y-1))
        if y < HEIGHT-1    and not visited[y+1][x]: candidates.append(('D', x, y+1))

        if candidates:
            direction, nx, ny = random.choice(candidates)

            # 壁を破壊
            if direction == 'L':  vert[y][x-1]   = False
            if direction == 'R':  vert[y][x]     = False
            if direction == 'U':  horiz[y-1][x]  = False
            if direction == 'D':  horiz[y][x]    = False

            visited[ny][nx] = True
            stack.append((nx, ny))
        else:
            stack.pop()

    return Maze(vert, horiz)

def render_maze(m: Maze) -> str:
    """壁を #, 通路を空白にして文字列化。"""
    w, h = WIDTH, HEIGHT
    cells = [['#'] * (2*w + 1) for _ in range(2*h + 1)]

    for y in range(h):
        for x in range(w):
            cx, cy = 2*x + 1, 2*y + 1
            cells[cy][cx] = ' '                       # セル本体

            # 左右上下の壁を確認し、無ければ空白を開通
            if x > 0 and not m.vert_walls[y][x-1]: cells[cy][cx-1] = ' '
            if x < w-1 and not m.vert_walls[y][x]: cells[cy][cx+1] = ' '
            if y > 0 and not m.horiz_walls[y-1][x]: cells[cy-1][cx] = ' '
            if y < h-1 and not m.horiz_walls[y][x]: cells[cy+1][cx] = ' '

    # スタートとゴールを目印付きで表示
    sx, sy = START; gx, gy = GOAL
    cells[2*sy+1][2*sx+1] = 'S'
    cells[2*gy+1][2*gx+1] = 'G'

    return '\n'.join(''.join(row) for row in cells)


In [14]:
if __name__ == "__main__":
    maze = generate_maze()
    printable = render_maze(maze)
    print(printable)


#####################
#   # #       #   # #
# # # # # ### # # # #
# #S#   #   #   #   #
# ######### ####### #
# #   #     #     # #
# # # # ##### ### # #
#   #   #     #   # #
######### ### # ### #
#       # #   # #   #
# # ##### # ##### ###
# #   #   #   #   # #
# ### # ##### # ### #
# # # # #     # #   #
# # # # # # ### # ###
#   #   # # #   #   #
### ##### ### ### # #
# #   # # #   #  G# #
# ### # # # ####### #
#       #           #
#####################


In [15]:
def render_maze_compact(m: Maze) -> str:
    """
    壁を次の文字で描画し、行数を (HEIGHT + 1) 行に縮小する。
    ────────────────────────────────────────────────
        '_' : 底辺（水平壁）
        '|' : 右辺（垂直壁）
        ' ' : 通路
        'S','G' : スタート / ゴール
    """
    w, h = WIDTH, HEIGHT
    # 上辺（外周）の水平壁
    lines = [' ' + '_' * w]           # 例)  " ____________"（先頭の空白は左上角の柱）

    for y in range(h):
        row = ['|']                   # 行頭は必ず左外壁
        for x in range(w):
            # デフォルトは通路
            char = ' '
            if (x, y) == START: char = 'S'
            elif (x, y) == GOAL: char = 'G'

            # 下側の水平壁を描画
            if y == h - 1:                            # 最下段は必ず外壁
                char = '_' if char == ' ' else char
            elif m.horiz_walls[y][x]:                 # 内部壁の場合
                char = '_' if char == ' ' else char

            # 右側の垂直壁を描画
            if x == w - 1:                            # 右外周は必ず壁
                row.append(char + '|')
            elif m.vert_walls[y][x]:                  # 内部壁
                row.append(char + '|')
            else:                                     # 壁が無い場合は空白
                row.append(char + ' ')
        lines.append(''.join(row))
    return '\n'.join(lines)


In [17]:
if __name__ == "__main__":
    maze = generate_maze()

    print("◆ 従来のフルスケール表示")
    print(render_maze(maze))

    print("\n◆ コンパクト表示")
    print(render_maze_compact(maze))


◆ 従来のフルスケール表示
#####################
#             #     #
# ########### # ### #
# #S        # #   # #
# ######### ### # # #
#         #   # # # #
# # # ####### ### # #
# # # #     #     # #
# # ### ### ####### #
# #     #       # # #
# ####### ##### # # #
#     #   #   #   # #
####### ##### ### # #
#       #   #     # #
# ####### # # ##### #
# #       # # #   # #
# # ####### # # # # #
# #   #   # # # #G  #
# ### # # # ### #####
#       # #         #
#####################

◆ コンパクト表示
 __________
|  _ _ _ _ _  |  _  |
| |S _ _ _  |_|   | |
|      _ _|_  |_| | |
| | |_|  _  |_ _ _| |
| |_ _ _|  _ _  | | |
|_ _ _|  _|_  |_  | |
|  _ _ _|   |  _ _| |
| |  _ _ _| | |   | |
| |_  |   | |_| |G _|
|_ _ _ _|_|_ _ _ _ _|


In [18]:
def generate_maze_limited() -> Maze:
    """
    直線が 5 マス以上続かないよう制御した DFS 版迷路生成。
    """
    visited = [[False]*WIDTH for _ in range(HEIGHT)]
    vert   = [[True]*(WIDTH-1) for _ in range(HEIGHT)]
    horiz  = [[True]*WIDTH      for _ in range(HEIGHT-1)]

    # スタック要素: (x, y, prev_dir, run_len)
    stack: List[Tuple[int, int, str, int]] = [(START[0], START[1], '', 0)]
    visited[START[1]][START[0]] = True

    while stack:
        x, y, prev_dir, run_len = stack[-1]

        # 未訪問の隣接セルを列挙
        neighbors = []
        if x > 0        and not visited[y][x-1]: neighbors.append(('L', x-1, y))
        if x < WIDTH-1  and not visited[y][x+1]: neighbors.append(('R', x+1, y))
        if y > 0        and not visited[y-1][x]: neighbors.append(('U', x, y-1))
        if y < HEIGHT-1 and not visited[y+1][x]: neighbors.append(('D', x, y+1))

        # 直線 5 マス禁止フィルタ
        if run_len >= 4 and neighbors:
            filtered = [nb for nb in neighbors if nb[0] != prev_dir]
            if filtered:                          # 少なくとも一方向残れば差し替え
                neighbors = filtered

        if neighbors:
            direction, nx, ny = random.choice(neighbors)

            # 壁破壊
            if direction == 'L':  vert[y][x-1]   = False
            if direction == 'R':  vert[y][x]     = False
            if direction == 'U':  horiz[y-1][x]  = False
            if direction == 'D':  horiz[y][x]    = False

            visited[ny][nx] = True
            next_run = run_len + 1 if direction == prev_dir else 1
            stack.append((nx, ny, direction, next_run))
        else:
            stack.pop()

    return Maze(vert, horiz)


In [19]:
if __name__ == "__main__":
    maze = generate_maze_limited()

    print("◆ 従来のフルスケール表示")
    print(render_maze(maze))

    print("\n◆ コンパクト表示")
    print(render_maze_compact(maze))

◆ 従来のフルスケール表示
#####################
#     #     #       #
# ##### ### ### ### #
# #S    # #   # # # #
# ####### ### # # # #
# #       # # #   # #
# # ### # # # ##### #
#   # # # # #       #
# ### # # # ####### #
#     # #         # #
##### # ### ####### #
# #   #   # #   #   #
# # ##### ### # # ###
# #   # #     #   # #
# ### # ########### #
#     #             #
# ##### ######### # #
# #     #     #  G# #
# ####### ### # ### #
#         #     #   #
#####################

◆ コンパクト表示
 __________
|  _ _|  _  |_   _  |
| |S _ _| |_  | | | |
| |  _    | | |_ _| |
|  _| | | | |_ _ _  |
|_ _  | |_   _ _ _| |
| |  _|_  |_|   |  _|
| |_  | |_ _ _|_ _| |
|  _ _|  _ _ _ _    |
| |_ _ _|  _  |  G| |
|_ _ _ _ _|_ _ _|_ _|
